#### Price Action Indicators

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
from finta import TA

import matplotlib as plot
import mplfinance as mpf
import plotly.graph_objects as go

#%matplotlib inline

database_name = "TSX_Quality.sqlite"

#### Load our Data  

In [ ]:
# Load from price table
# Convert string columns to float values
# Replace "-" prices with Close values
# Make sure al dates are sorted in ascending order (TA libraries need this order)

In [2]:
# Heavy function to retreive all prices from database (more than 6 million rows, takes many seconds to execute)
def get_all_prices():
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM Prices_Daily ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True)

    prices_per_ticker = prices.groupby(["Ticker"])

    return prices_per_ticker

def get_prices_for(symbol):
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM Prices_Daily WHERE Ticker = '{symbol}' ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True) 
    
    return prices   

def get_all_symbols():
    conn = sqlite3.connect(database_name)
    sql = f"SELECT * FROM Symbols ORDER BY UPPER(Ticker) ASC"
    symbols_df = pd.read_sql_query(sql, conn)
    
    return symbols_df

#prices_df = get_all_prices()
# Access specific company using .get_group("TickerSymbol")


In [4]:
# prices_per_ticker.get_group("SHOP").loc["2021-03" : "2021-05"]   #Deprecated method sort dataframe before using slicing on index
# prices_per_ticker.get_group("SHOP").sort_index().loc["2016-03" : "2016-05"]
#prices_df.get_group("SHOP")

In [ ]:
# prices_per_ticker.get_group("SHOP").sort_index().loc["2015"]["High"].max()
#prices_df.get_group("SHOP").sort_index().loc["2020"]["low"].min()
# 2228.73  : MAX All years      MIN   25.85
# 1742.37  : MAX 2022           MIN 1424.95
# 2228.73  : MAX 2021           MIN 1248.55
# 1658.79  : MAX 2020           MIN  435.03
#  544.00  : MAX 2019           MIN  174.59
#  232.65  : MAX 2018           MIN  126.65
#  151.88  : MAX 2017           MIN   56.60
#   60.93  : MAX 2016           MIN   25.85
#   53.50  : MAX 2015           MIN   30.00

In [ ]:
# prices_2 = prices_df.get_group("SHOP").copy()
# prices_2["SMA"] = prices_2["close"].rolling(window=5).mean()
# prices_2.tail(5)

In [38]:
def run_indicators(prices):
    data_df = prices.copy()
    symbol = data.iloc[0]["Ticker"]
    data_df.drop("Ticker", axis=1, inplace=True)
    #data_df.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    #data_df.index.rename("date", inplace=True)
    #data_df.sort_index(ascending=True, inplace=True)
    
    indicators_df = pd.DataFrame()
    # Return a series
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"SMA{period}"] = TA.SMA(data_df, int(period))
    except Exception:
        print(f"Error calculating SMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"EMA{period}"] = TA.EMA(data_df, int(period))
    except Exception:
        print(f"Error calculating EMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"VAMA{period}"] = TA.VAMA(data_df, int(period))
    except Exception:
        print(f"Error calculating VAMA{period}")
    try:
        for period in ["200", "100", "50", "30", "20", "15", "9", "5"]:
            indicators_df[f"WMA{period}"] = TA.WMA(data_df, int(period))
    except Exception:
        print(f"Error calculating WMA{period}")
    try:
        for period in ["30", "20", "14", "10", "7"]:
            indicators_df[f"RSI{period}"] = TA.RSI(data_df, int(period))
    except Exception:
        print(f"Error calculating RSI{period}")
    try:
        for period in ["30", "20", "14", "10", "7"]:
            indicators_df[f"ATR{period}"] = TA.ATR(data_df, int(period))
    except Exception:
        print(f"Error calculating ATR{period}")        

    data_df = data_df.join(indicators_df)

    try:
        data_df["UpTrend"] = data_df["close"] > data_df["SMA200"]
        data_df["DownTrend"] = data_df["close"] <= data_df["SMA200"]
    except Exception:
        data_df["UpTrend"] = False
        data_df["DownTrend"] = False
        print(f"{symbol} : Error identifying UP/Down Trend")

    data_df.insert(0,"ticker", symbol, allow_duplicates=True)
    return data_df


# data = prices_df.get_group("AFN.DB.G").copy()
# data.drop( data.loc[data["volume"] == 0].index, inplace=True )
# run_indicators(data).tail(5)


In [ ]:
def run_other_indicators(prices):
    data_df = prices.copy()
    symbol = data.iloc[0]["Ticker"]
    data_df.drop("Ticker", axis=1, inplace=True)   
    
    indicators_df = pd.DataFrame()
    # Returns a Dataframe
    try:
        MACD = TA.MACD(data_df)
        MACD.rename(columns={"SIGNAL":"MACD_SIGNAL"}, inplace=True)
        data_df = data_df.join(MACD)
    except Exception:
        data_df["MACD"] = np.NaN
        data_df["MACD_SIGNAL"] = np.NaN
        print(f"Error calculating MACD")

    try:
        VW_MACD = TA.MACD(data_df)
        VW_MACD.rename(columns={"MACD":"VW_MACD"}, inplace=True)
        VW_MACD.rename(columns={"SIGNAL":"VW_MACD_SIGNAL"}, inplace=True)    
        data_df = data_df.join(VW_MACD)
    except Exception:
        data_df["VW_MACD"] = np.NaN
        data_df["VW_MACD_SIGNAL"] = np.NaN
        print(f"Error calculating VW_MACD (Value Weighted MACD)")

    try:
        BBANDS = TA.BBANDS(data_df) # Standard 20 day period, with 2 standard deviations
        data_df = data_df.join(BBANDS)
    except Exception:
        data_df["BB_UPPER"]  = np.NaN
        data_df["BB_MIDDLE"] = np.NaN
        data_df["BB_LOWER"]  = np.NaN
        print(f"Error calculating BBANDS (Bollinger Bands)")


In [18]:
symbols = get_all_symbols()
print(symbols.shape)
list1 = symbols[1:101]["ticker"].tolist()
str1 = "("
for s in list1:
    str1 += f"'{s}',"
str1 += ")"   
print(str1)

(4330, 3)
('A','AAA.P','AAB','AAC.P','AAD.P','AAG','AALI','AAN','AAT','AAU','AAV','AAZ','ABCT','ABI','ABN','ABR','ABRA','ABST','ABTC','ABTC.U','ABX','ABZ','AC','ACB','ACB.WT.U','ACB.WT.V','ACCA.P','ACD','ACD.DB','ACO.X','ACO.Y','ACOG','ACOG.WT','ACOP','ACP','ACQ','ACS','ACST','ACU','ACZ','AD.DB','AD.UN','ADCO','ADCO.WT','ADD','ADE','ADG','ADK','ADN','ADW.A','ADW.B','ADY','ADYA','ADZ','ADZN','AE','AEC','AEG','AEM','AEP','AEX','AEZS','AF.P','AFE','AFF','AFM','AFN','AFN.DB.D','AFN.DB.E','AFN.DB.F','AFN.DB.G','AFN.DB.H','AFN.DB.I','AFR','AGC','AGD','AGF.B','AGG','AGI','AGL','AGLD','AGO','AGR.UN','AGX','AH','AH.DB','AH.WT','AH.WT.A','AH.WT.B','AHC','AHC.WT','AHR','AI','AI.DB.C','AI.DB.D','AI.DB.E','AI.DB.F','AIF','AII','AIM',)


In [39]:
def strategy_1(prices):
    """ Rayner Teo : https://www.youtube.com/watch?v=W8ENIXvcGlQ
    CONDITIONS:
    1) Close price > SMA200
    2) RSI10 < 30
    3) Next Day = BUY
    """
    
    for i in prices.index:
        print(f"{i}")

    return True

# conn = sqlite3.connect(database_name)
# symbol = "SHOP"
# SQL = f"SELECT * FROM Prices_Daily WHERE ticker ='{symbol}' ORDER BY Date ASC"
# data = pd.read_sql_query(SQL, conn)

symbols = get_all_symbols()
list1 = symbols[1:101]["ticker"].tolist()

for symbol in list1:
    data = get_prices_for(symbol)
    indicators = run_indicators(data)
    
    conn = sqlite3.connect(database_name)
    indicators.to_sql("Indicators", conn, if_exists="append")
    print(f"{symbol} : Saved indicators to DB - {data.shape}")


A : Saved indicators to DB - (2520, 6)
AAA.P : Saved indicators to DB - (84, 6)
AAB : Saved indicators to DB - (2520, 6)
AAC.P : Saved indicators to DB - (189, 6)
AAD.P : Saved indicators to DB - (97, 6)
AAG : Saved indicators to DB - (2520, 6)
AALI : Saved indicators to DB - (2520, 6)
AAN : Saved indicators to DB - (2520, 6)
AAT : Saved indicators to DB - (2520, 6)
AAU : Saved indicators to DB - (2519, 6)
AAV : Saved indicators to DB - (2520, 6)
AAZ : Saved indicators to DB - (2366, 6)
ABCT : Saved indicators to DB - (228, 6)
ABI : Saved indicators to DB - (2520, 6)
ABN : Saved indicators to DB - (2520, 6)
ABR : Saved indicators to DB - (2225, 6)
ABRA : Saved indicators to DB - (2520, 6)
ABST : Saved indicators to DB - (2520, 6)
ABTC : Saved indicators to DB - (95, 6)
ABTC.U : Saved indicators to DB - (24, 6)
ABX : Saved indicators to DB - (2520, 6)
ABZ : Saved indicators to DB - (2372, 6)
AC : Saved indicators to DB - (2520, 6)
ACB : Saved indicators to DB - (1327, 6)
ACB.WT.U : Save

In [37]:
data_test = get_prices_for("AFN.DB.G")
# data_test    # 609 rows
mask = data_test["volume"] < 1
# data_test.loc[mask]  # 67 to drop

data_test.drop(data_test.loc[mask].index, inplace=True)
data_test.dropna
data_test.resample("24H")
data_test
indic = TA.MACD(data_test)
indic.rename(columns={"SIGNAL":"MACD_SIGNAL"}, inplace=True)
indic



,MACD,MACD_SIGNAL
date,,
2019-11-19,0.000000,0.000000
2019-11-19,0.000000,0.000000
2019-11-19,0.000000,0.000000
2019-11-19,0.000000,0.000000
2019-11-19,0.000000,0.000000
...,...,...
2022-01-10,0.037712,-0.125443
2022-01-11,0.134242,-0.073506
2022-01-12,0.154893,-0.027827
